In [1]:
# **STEP 0: IMPORT LIBRARIES**
import yfinance as yf
import pandas as pd
import os

# **STEP 1: DEFINE FOREX PAIRS LIST**
# Danh sách các mã Forex phổ biến. Bạn có thể bổ sung thêm nhiều cặp khác từ các nguồn tài chính.
forex_pairs = ['EURUSD=X', 'USDJPY=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'EURJPY=X']  # Thêm các mã khác vào đây nếu cần

# Tạo thư mục để lưu dữ liệu nếu nó chưa tồn tại
directory = 'Forex_Data'
if not os.path.exists(directory):
    os.makedirs(directory)

# **STEP 2: DOWNLOAD AND SAVE HISTORICAL DATA FOR EACH PAIR**
# Tải dữ liệu lịch sử cho mỗi mã Forex và lưu vào tệp CSV
for pair in forex_pairs:
    print(f"Downloading data for {pair}...")
    data = yf.download(pair, period="max", interval='1d')  # Tải dữ liệu từ khi cặp tiền tệ xuất hiện
    data.reset_index(inplace=True)  # Reset index để đảm bảo cột Date là một cột bình thường
    
    # Lưu dữ liệu vào file CSV
    file_path = os.path.join(directory, f'{pair}_data.csv')
    data.to_csv(file_path, index=False)
    print(f"Saved data for {pair} at {file_path}")

# **STEP 3: MERGE ALL FOREX DATA INTO ONE DATAFRAME**
# Đọc tất cả dữ liệu đã lưu và kết hợp chúng lại thành một DataFrame duy nhất

# Khởi tạo một DataFrame trống
merged_data = pd.DataFrame()

for pair in forex_pairs:
    file_path = os.path.join(directory, f'{pair}_data.csv')
    
    # Đọc dữ liệu từ file CSV
    data = pd.read_csv(file_path)
    
    # Đổi tên cột 'Close' để phân biệt giữa các cặp tiền
    data.rename(columns={'Close': f'{pair}_Close'}, inplace=True)
    
    # Chỉ lấy cột 'Date' và 'Close' để kết hợp
    if merged_data.empty:
        merged_data = data[['Date', f'{pair}_Close']]  # Lần đầu tiên, lấy cả 'Date' và 'Close'
    else:
        merged_data = pd.merge(merged_data, data[['Date', f'{pair}_Close']], on='Date', how='outer')  # Ghép dựa trên cột 'Date'

# **STEP 4: HANDLE MISSING DATA**
# Xử lý các giá trị bị thiếu (có thể có vì một số cặp không có dữ liệu tại mọi thời điểm)
merged_data.fillna(method='ffill', inplace=True)  # Điền giá trị thiếu bằng cách sử dụng giá trị gần nhất
merged_data.fillna(method='bfill', inplace=True)  # Điền giá trị thiếu bằng cách sử dụng giá trị kế tiếp nếu có

# Hiển thị dữ liệu đã kết hợp
print(merged_data.head())

# **STEP 5: SAVE THE MERGED DATA**
# Lưu dữ liệu đã kết hợp vào một file CSV
merged_file_path = os.path.join(directory, 'merged_forex_data.csv')
merged_data.to_csv(merged_file_path, index=False)
print(f"Saved merged Forex data at {merged_file_path}")

# **STEP 6: PREPARE DATA FOR MODEL TRAINING**
# Ở bước này, ta có thể sử dụng các cột giá đóng cửa của các mã khác nhau làm đặc trưng để dự đoán
# Cặp tiền chính mà ta muốn dự đoán ở đây là EURJPY

# Tạo biến X (đặc trưng) và y (mục tiêu)
X = merged_data.drop(columns=['Date', 'EURJPY=X_Close'])  # Sử dụng các cặp tiền khác làm đặc trưng
y = merged_data['EURJPY=X_Close']  # Mục tiêu là giá đóng cửa của EUR/JPY

# **STEP 7: SPLIT THE DATA FOR TRAINING AND TESTING**
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **STEP 8: TRAIN A RANDOM FOREST MODEL**
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm thử
y_pred = model.predict(X_test)

# Tính toán lỗi bình phương trung bình (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# **STEP 9: VISUALIZE THE PREDICTION RESULTS**
import matplotlib.pyplot as plt

# Vẽ biểu đồ so sánh giá thực và giá dự đoán của EUR/JPY
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual EUR/JPY Prices', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted EUR/JPY Prices', color='orange')
plt.title('EUR/JPY Price Prediction Based on Other Forex Pairs')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# **STEP 10: SAVE THE MODEL**
import joblib

# Lưu mô hình đã huấn luyện
model_path = os.path.join(directory, 'forex_model.pkl')
joblib.dump(model, model_path)
print(f"Model saved at {model_path}")


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EURUSD=X']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2023-08-20 -> 2023-09-19) (Yahoo error = "1m data not available for startTime=1692486000 and endTime=1695078000. Only 7 days worth of 1m granularity data are allowed to be fetched per request.")')


Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume]
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.